Chromosome Alignment 

- Circos: https://dash.plotly.com/dash-bio/circos 
- Clustegram:  https://dash.plotly.com/dash-bio/clustergram 
- FornaContainer: https://dash.plotly.com/dash-bio/fornacontainer 
- Ideogram: https://dash.plotly.com/dash-bio/ideogram 
- Lgv:  https://dash.plotly.com/dash-bio/igv 
- Manhattan Plot:  https://dash.plotly.com/dash-bio/manhattanplot 


In [1]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import re

## Circos
https://dash.plotly.com/dash-bio/circos

In [4]:
import json
import six.moves.urllib.request as urlreq
from six import PY3

import dash
import dash_bio as dashbio
import dash_html_components as html
import dash_core_components as dcc


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

data = urlreq.urlopen(
    'https://raw.githubusercontent.com/plotly/dash-bio-docs-files/master/' +
    'circos_graph_data.json'
).read()

if PY3:
    data = data.decode('utf-8')

circos_graph_data = json.loads(data)

app.layout = html.Div([
    dashbio.Circos(
        id='my-dashbio-circos',
        layout=circos_graph_data['GRCh37'],
        selectEvent={"0": "hover", "1": "click", "2": "both"},
        tracks=[{
            'type': 'CHORDS',
            'data': circos_graph_data['chords'],
            'config': {
                'tooltipContent': {
                    'source': 'source',
                    'sourceID': 'id',
                    'target': 'target',
                    'targetID': 'id',
                    'targetEnd': 'end'
                }
            }
        }]
    ),
    "Graph type:",
    dcc.Dropdown(
        id='histogram-chords',
        options=[
            {'label': x, 'value': x}
            for x in ['histogram', 'chords']
        ],
        value='chords'
    ),
    "Event data:",
    html.Div(id='circos-output')
])


@app.callback(
    dash.dependencies.Output('circos-output', 'children'),
    [dash.dependencies.Input('my-dashbio-circos', 'eventDatum')]
)
def update_output(value):
    if value is not None:
        return [html.Div('{}: {}'.format(v.title(), value[v]))
                for v in value.keys()]
    return 'There are no event data. Click or hover on a data point to get more information.'


@app.callback(
    dash.dependencies.Output('my-dashbio-circos', 'tracks'),
    [dash.dependencies.Input('histogram-chords', 'value')],
    state=[dash.dependencies.State('my-dashbio-circos', 'tracks')]
)
def change_graph_type(value, current):

    if value == 'histogram':

        current[0].update(
            data=circos_graph_data['histogram'],
            type='HISTOGRAM'
        )

    elif value == 'chords':
        current[0].update(
            data=circos_graph_data['chords'],
            type='CHORDS',
            config={
                'tooltipContent': {
                    'source': 'source',
                    'sourceID': 'id',
                    'target': 'target',
                    'targetID': 'id',
                    'targetEnd': 'end'
                }
            }
        )
    return current


if __name__ == '__main__':
    app.run_server("inline")

## Clustegram
https://dash.plotly.com/dash-bio/clustergram 

In [3]:
import pandas as pd

import dash
import dash_bio as dashbio
import dash_html_components as html
import dash_core_components as dcc


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv(
    'https://raw.githubusercontent.com/plotly/dash-bio-docs-files/master/' +
    'clustergram_mtcars.tsv',
    sep='	', skiprows=4
).set_index('model')

columns = list(df.columns.values)
rows = list(df.index)

app.layout = html.Div([
    "Rows to display",
    dcc.Dropdown(
        id='clustergram-input',
        options=[
            {'label': row, 'value': row} for row in list(df.index)
        ],
        value=rows[:10],
        multi=True
    ),

    html.Div(
        id='my-clustergram'
    )
])


@app.callback(
    dash.dependencies.Output('my-clustergram', 'children'),
    [dash.dependencies.Input('clustergram-input', 'value')]
)
def update_clustergram(rows):
    if len(rows) < 2:
        return "Please select at least two rows to display."

    return dcc.Graph(figure=dashbio.Clustergram(
        data=df.loc[rows].values,
        column_labels=columns,
        row_labels=rows,
        color_threshold={
            'row': 250,
            'col': 700
        },
        hidden_labels='row',
        height=800,
        width=700
    ))


if __name__ == '__main__':
    app.run_server("inline")

## FornaContainer
https://dash.plotly.com/dash-bio/fornacontainer 

In [5]:
import dash
import dash_bio as dashbio
import dash_core_components as dcc
import dash_html_components as html
from dash.exceptions import PreventUpdate

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

sequences = {
    'PDB_01019': {
        'sequence': 'AUGGGCCCGGGCCCAAUGGGCCCGGGCCCA',
        'structure': '.((((((())))))).((((((()))))))'
    },
    'PDB_00598': {
        'sequence': 'GGAGAUGACgucATCTcc',
        'structure': '((((((((()))))))))'
    }
}

app.layout = html.Div([
    dashbio.FornaContainer(
        id='forna'
    ),
    html.Hr(),
    html.P('Select the sequences to display below.'),
    dcc.Dropdown(
        id='forna-sequence-display',
        options=[
            {'label': name, 'value': name} for name in sequences.keys()
        ],
        multi=True,
        value=['PDB_01019']
    )
])


@app.callback(
    dash.dependencies.Output('forna', 'sequences'),
    [dash.dependencies.Input('forna-sequence-display', 'value')]
)
def show_selected_sequences(value):
    if value is None:
        raise PreventUpdate
    return [
        sequences[selected_sequence]
        for selected_sequence in value
    ]


if __name__ == '__main__':
    app.run_server("inline")


## Ideogram
https://dash.plotly.com/dash-bio/ideogram 

In [6]:
import dash
import dash_bio as dashbio
import dash_html_components as html
import dash_core_components as dcc

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    'Select which chromosomes to display on the ideogram below:',
    dcc.Dropdown(
        id='displayed-chromosomes',
        options=[{'label': str(i), 'value': str(i)} for i in range(1, 23)],
        multi=True,
        value=[str(i) for i in range(1, 23)]
    ),
    dashbio.Ideogram(
        id='my-dashbio-ideogram'
    ),
    html.Div(id='ideogram-rotated')
])


@app.callback(
    dash.dependencies.Output('my-dashbio-ideogram', 'chromosomes'),
    [dash.dependencies.Input('displayed-chromosomes', 'value')]
)
def update_ideogram(value):
    return value


@app.callback(
    dash.dependencies.Output('ideogram-rotated', 'children'),
    [dash.dependencies.Input('my-dashbio-ideogram', 'rotated')]
)
def update_ideogram_rotated(rot):
    return 'You have {} selected a chromosome.'.format(
        '' if rot else 'not')


if __name__ == '__main__':
    app.run_server("inline")

    
## Lgv
https://dash.plotly.com/dash-bio/igv 




In [7]:
import dash
import dash_bio as dashbio
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

HOSTED_GENOME_DICT = [
    {'value': 'mm10', 'label': 'Mouse (GRCm38/mm10)'},
    {'value': 'rn6', 'label': 'Rat (RGCS 6.0/rn6)'},
    {'value': 'gorGor4', 'label': 'Gorilla (gorGor4.1/gorGor4)'},
    {'value': 'panTro4', 'label': 'Chimp (SAC 2.1.4/panTro4)'},
    {'value': 'panPan2', 'label': 'Bonobo (MPI-EVA panpan1.1/panPan2)'},
    {'value': 'canFam3', 'label': 'Dog (Broad CanFam3.1/canFam3)'},
    {'value': 'ce11', 'label': 'C. elegans (ce11)'}
]

app.layout = html.Div([
    dcc.Loading(
        id='igv-container'
    ),
    html.Hr(),
    html.P('Select the genome to display below.'),
    dcc.Dropdown(
        id='igv-genome-select',
        options=HOSTED_GENOME_DICT,
        value='ce11'
    )
])



# Return the IGV component with the selected genome.
@app.callback(
    Output('igv-container', 'children'),
    Input('igv-genome-select', 'value')
)
def return_igv(genome):
    return (
        html.Div([
            dashbio.Igv(
                id='default-igv',
                genome=genome,
                minimumBases=100,
            )
        ])
    )


if __name__ == '__main__':
    app.run_server("inline")

## Manhattan Plot
https://dash.plotly.com/dash-bio/manhattanplot 



In [ ]:
import pandas as pd

import dash
import dash_bio as dashbio
import dash_html_components as html
import dash_core_components as dcc


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv(
    'https://raw.githubusercontent.com/plotly/dash-bio-docs-files/master/' +
    'manhattan_data.csv'
)

app.layout = html.Div([
    'Threshold value',
    dcc.Slider(
        id='manhattanplot-input',
        min=1,
        max=10,
        marks={
            i: {'label': str(i)} for i in range(10)
        },
        value=6
    ),
    html.Br(),
    html.Div(
        dcc.Graph(
            id='my-dashbio-manhattanplot',
            figure=dashbio.ManhattanPlot(
                dataframe=df
            )
        )
    )
])


@app.callback(
    dash.dependencies.Output('my-dashbio-manhattanplot', 'figure'),
    [dash.dependencies.Input('manhattanplot-input', 'value')]
)
def update_manhattanplot(threshold):

    return dashbio.ManhattanPlot(
        dataframe=df,
        genomewideline_value=threshold
    )


if __name__ == '__main__':
    app.run_server(debug=True)